## 기본설정 및 함수정의

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
from shapely.ops import unary_union
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

# 영등포역 위도, 경도
ydp = [37.51569886583137, 126.90768824079629]

# Point를만드는 함수
def make_point(x):
    try:
        return Point(x)
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])
    
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

#### 영등포구 행정경계 (출처 - 통계지리정보서비스 2023년 센서스용 행정구역경계(읍면동))

In [2]:
# shp to geodataframe convert
shapefile_path = "SBJ_2406_001/_census_data_2023_bnd_dong_bnd_dong_11190_2023_2023\\bnd_dong_11190_2023_2023_2Q.shp"
ydp_gdf = gpd.read_file(shapefile_path)
ydp_gdf = ydp_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환

#### 격자(매핑용)

In [3]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_001/19.영등포_격자(매핑용).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))
grid_map_df.drop(columns="geometry.coordinates", axis=1, inplace=True)
# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')

#### 영등포구 상세 도로망

In [4]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_001/6.영등포_상세도로망.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
roadsystem_df = pd.json_normalize(geojson_data['features'])
roadsystem_df['geometry'] = roadsystem_df['geometry.coordinates'].apply(lambda x : make_lin(x))
roadsystem_df['properties.link_id'] = roadsystem_df['properties.link_id'].astype(str)
roadsystem_df.drop(columns="geometry.coordinates", axis=1, inplace=True)
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
roadsystem_df = gpd.GeoDataFrame(roadsystem_df, geometry='geometry')

#### 도로명주소(도로)

In [5]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_001/7.영등포_도로명주소(도로).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
roadname_df = pd.json_normalize(geojson_data['features'])
roadname_df['geometry.coordinates'] = roadname_df['geometry.coordinates'].apply(lambda x : make_lin(x))
roadname_df.rename(columns={'geometry.coordinates' : 'geometry'}, inplace=True)
# roadname_df 데이터프레임을 GeoDataFrame으로 변환
roadname_df = gpd.GeoDataFrame(roadname_df, geometry='geometry')

#### 소방용수시설 시각화

In [7]:
# 소방용수시설 데이터프레임을 GeoPandas 데이터프레임으로 변환
firefighting_water_df = pd.read_csv('SBJ_2406_001/22.영등포_소방용수시설.csv')

#### 소방서 위치정보

In [8]:
# 소방서 데이터프레임을 GeoPandas 데이터프레임으로 변환
firehouse_df = pd.read_csv('SBJ_2406_001/23.영등포_소방서위치정보.csv')

In [ ]:
# 맵 객체 생성 및 데이터 로드
ydp_firefacility_map = KeplerGl(height=1000, width=1500)
ydp_firefacility_map.add_data(data=firehouse_df, name="영등포구 소방서 데이터")
ydp_firefacility_map.add_data(data=firefighting_water_df, name="영등포구 소방용수시설 데이터")
ydp_firefacility_map.add_data(data=ydp_gdf, name="영등포구 행정동 데이터")
ydp_firefacility_map.add_data(data=roadname_df, name="영등포구 도로망 데이터")

# 맵 출력 및 상세설정
ydp_firefacility_map

In [30]:
# 맵 저장
ydp_firefacility_map.save_to_html(file_name="visualization/영등포구 건물 및 소방관련/영등포구 소방시설 시각화 map.html")

Map saved to visualization/영등포구 건물 및 소방관련/영등포구 소방시설 시각화 map.html!


#### 도로명주소(건물)

In [16]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_001/13.영등포_GIS건물일반집합정보.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
gbuilding_df = pd.json_normalize(geojson_data['features'])
gbuilding_df['properties.emd_nm'] = gbuilding_df['properties.emd_nm'].apply(lambda x : x.split(" ")[-1])
gbuilding_df

,type,properties.pnu,properties.emd_cd,properties.emd_nm,properties.s_land_cd,properties.s_land_nm,properties.jibun,properties.bldg_nm,properties.bldg_dong,properties.mn_sb_cd,properties.mn_sb_nm,properties.plot_area,properties.bldg_area,properties.gfa,properties.vl_rat,properties.bc_rat,properties.bldg_st_cd,properties.bldg_st_nm,properties.bdtyp_cd,properties.bdtyp_nm,properties.bldg_hght,properties.grnd_flr,properties.ugrnd_flr,geometry.type,geometry.coordinates
0,Feature,1156013200103550631,1156013200,신길동,1,일반,355-631,None,None,0,주건축물,0.0,64.92,350.66,0.00000,0.00,21,철근콘크리트구조,03000,제1종근린생활시설,0.0,5.0,1.0,Polygon,"[[[126.90260903842993, 37.503751666785355], [1..."
1,Feature,1156013200100950086,1156013200,신길동,1,일반,95-86,None,None,0,주건축물,89.0,50.91,245.13,227.39000,57.20,21,철근콘크리트구조,03000,제1종근린생활시설,13.4,4.0,1.0,Polygon,"[[[126.9195237482927, 37.51369537304772], [126..."
2,Feature,1156011200100110005,1156011200,당산동2가,1,일반,11-5,None,None,0,주건축물,0.0,213.92,1339.07,0.00000,0.00,21,철근콘크리트구조,03000,제1종근린생활시설,0.0,5.0,1.0,Polygon,"[[[126.8944849620747, 37.52094224849279], [126..."
3,Feature,1156013300109760047,1156013300,대림동,1,일반,976-47,None,None,0,주건축물,760.0,435.86,1971.43,168.37000,57.35,21,철근콘크리트구조,09000,의료시설,15.2,4.0,2.0,Polygon,"[[[126.90668087175683, 37.49050985273767], [12..."
4,Feature,1156013200115510000,1156013200,신길동,1,일반,1551,이조빌딩,None,0,주건축물,140.0,83.46,259.86,185.61000,59.61,21,철근콘크리트구조,01000,단독주택,17.3,5.0,0.0,Polygon,"[[[126.91718042846131, 37.50141921930075], [12..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18436,Feature,1156013300107100010,1156013300,대림동,1,일반,710-10,None,None,0,주건축물,1323.0,659.94,4275.20,199.32000,49.88,21,철근콘크리트구조,03000,제1종근린생활시설,21.0,5.0,2.0,Polygon,"[[[126.89958151253353, 37.494801186436725], [1..."
18437,Feature,1156010400100070026,1156010400,영등포동3가,1,일반,7-26,영등포모텔,None,0,주건축물,413.3,247.72,2093.97,494.59714,59.94,21,철근콘크리트구조,15000,숙박시설,33.1,10.0,1.0,Polygon,"[[[126.90655957513583, 37.519006891163805], [1..."
18438,Feature,1156010900100120001,1156010900,영등포동8가,1,일반,12-1,None,None,0,주건축물,0.0,284.07,1477.69,0.00000,0.00,21,철근콘크리트구조,09000,의료시설,0.0,5.0,1.0,Polygon,"[[[126.90622680128975, 37.52602038300536], [12..."
18439,Feature,1156012800100030001,1156012800,양평동4가,1,일반,3-1,코틴빌딩,None,0,주건축물,453.2,270.09,1883.36,291.45000,59.60,21,철근콘크리트구조,04000,제2종근린생활시설,21.6,5.0,2.0,Polygon,"[[[126.89707386332108, 37.53443900808547], [12..."


In [29]:
gbuilding_df['properties.emd_nm'].unique()

array(['신길동', '당산동2가', '대림동', '영등포동7가', '여의도동', '양평동2가', '문래동3가', '도림동',
       '문래동4가', '문래동2가', '당산동3가', '양평동3가', '양평동1가', '당산동', '영등포동4가',
       '영등포동2가', '당산동4가', '영등포동3가', '문래동6가', '양평동6가', '영등포동6가', '영등포동8가',
       '양평동5가', '양평동4가', '문래동5가', '영등포동', '당산동1가', '영등포동5가', '당산동5가',
       '당산동6가', '문래동1가', '양화동', '영등포동1가', '양평동'], dtype=object)

In [32]:
set(pyo_building_df['plot_lctn'].unique()) - set(gbuilding_df['properties.emd_nm'].unique())

{'산', '영등포구'}

In [28]:
pyo_building_df['plot_lctn'].unique()

array(['영등포동', '산', '영등포동1가', '영등포동2가', '영등포동3가', '영등포동4가', '영등포동5가',
       '영등포동6가', '영등포구', '영등포동7가', '영등포동8가', '여의도동', '당산동1가', '당산동2가',
       '당산동3가', '당산동4가', '당산동5가', '당산동6가', '당산동', '도림동', '문래동1가', '문래동2가',
       '문래동3가', '문래동4가', '문래동5가', '문래동6가', '양평동1가', '양평동2가', '양평동3가',
       '양평동4가', '양평동5가', '양평동6가', '양화동', '신길동', '대림동', '양평동'],
      dtype=object)

In [34]:
pyo_building_df[pyo_building_df['plot_lctn'] == '영등포구']

,plot_lctn,sgg_cd,emd_cd,plot_gbn,bun,ji,bd_rgst_pk,bd_rgst_nm,bd_rgst_type,bd_nm,dong_nm,mos_nm,bd_area,y_area,tdy_area,strct_nm,use_nm,etc_use_nm,hght,grnd_flr_cnt,ugrnd_flr_cnt,y_area.1,qk_prf_ny,qk_prf_ablty
3296,영등포구,11560,10800,0,0,0,11560-6269,일반,일반건축물,NaN,NaN,주건축물,173.74,521.22,521.22,철근콘크리트구조,제1종근린생활시설,창고시설,11.1,3,0,19960710.0,N.N,NaN


In [36]:
pyo_building_df.iloc[3296]

plot_lctn        서울특별시 영등포구 영등포동7가
sgg_cd                       11560
emd_cd                       10800
plot_gbn                         0
bun                              0
ji                               0
bd_rgst_pk              11560-6269
bd_rgst_nm                      일반
bd_rgst_type                 일반건축물
bd_nm                          NaN
dong_nm                        NaN
mos_nm                        주건축물
bd_area                     173.74
y_area                      521.22
tdy_area                    521.22
strct_nm                  철근콘크리트구조
use_nm                   제1종근린생활시설
etc_use_nm                    창고시설
hght                          11.1
grnd_flr_cnt                     3
ugrnd_flr_cnt                    0
y_area.1                19960710.0
qk_prf_ny                      N.N
qk_prf_ablty                   NaN
Name: 3296, dtype: object

In [35]:
pyo_building_df = pd.read_csv('SBJ_2406_001/14.영등포_건축물대장_표제부.csv')
#pyo_building_df['plot_lctn'] = pyo_building_df['plot_lctn'].apply(lambda x : x.split(" 영등포구 ")[-1])
#pyo_building_df['plot_lctn'] = pyo_building_df['plot_lctn'].apply(lambda x : x.split(" ")[-2])
pyo_building_df

,plot_lctn,sgg_cd,emd_cd,plot_gbn,bun,ji,bd_rgst_pk,bd_rgst_nm,bd_rgst_type,bd_nm,dong_nm,mos_nm,bd_area,y_area,tdy_area,strct_nm,use_nm,etc_use_nm,hght,grnd_flr_cnt,ugrnd_flr_cnt,y_area.1,qk_prf_ny,qk_prf_ablty
0,서울특별시 영등포구 영등포동 111번지,11560,10100,0,111,0,11560-2135,일반,일반건축물,NaN,NaN,주건축물,51.90,51.90,51.90,일반목구조,단독주택,주택,0.0,1,0,19601015.0,NaN,NaN
1,서울특별시 영등포구 영등포동 121번지,11560,10100,0,121,0,11560-2136,일반,일반건축물,NaN,NaN,주건축물,13.20,13.20,13.20,일반목구조,단독주택,납옥,0.0,1,0,19270921.0,NaN,NaN
2,서울특별시 영등포구 영등포동 125번지,11560,10100,0,125,0,11560-2137,일반,일반건축물,NaN,NaN,주건축물,16.53,16.53,16.53,일반목구조,단독주택,주택,0.0,1,0,19761006.0,NaN,NaN
3,서울특별시 영등포구 영등포동 135번지,11560,10100,0,135,0,11560-2138,일반,일반건축물,NaN,NaN,주건축물,80.66,80.66,80.66,일반목구조,단독주택,주택,0.0,1,0,19601115.0,NaN,NaN
4,서울특별시 영등포구 영등포동 144-14번지,11560,10100,0,144,14,11560-2139,일반,일반건축물,NaN,NaN,주건축물,35.70,35.70,35.70,블록구조,단독주택,주택,0.0,1,0,19670510.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27752,서울특별시 영등포구 양평동 82번지,11560,13400,0,82,0,11560-35496,일반,일반건축물,NaN,NaN,주건축물,386.77,386.77,386.77,일반목구조,제2종근린생활시설,"사무실, 창고",0.0,1,0,NaN,NaN,NaN
27753,서울특별시 영등포구 양평동 88번지,11560,13400,0,88,0,11560-35497,일반,일반건축물,NaN,NaN,주건축물,26.45,26.45,26.45,일반목구조,단독주택,주택,0.0,1,0,NaN,NaN,NaN
27754,서울특별시 영등포구 양평동 88번지,11560,13400,0,88,0,11560-35498,일반,일반건축물,NaN,NaN,주건축물,36.36,36.36,36.36,일반목구조,단독주택,주택,0.0,1,0,NaN,NaN,NaN
27755,서울특별시 영등포구 양평동 89-1번지,11560,13400,0,89,1,11560-35499,일반,일반건축물,NaN,NaN,주건축물,62.81,62.81,62.81,일반목구조,단독주택,주택,0.0,1,0,NaN,NaN,NaN


In [9]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_001/11.영등포_도로명주소(건물).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
building_df = pd.json_normalize(geojson_data['features'])
building_df['geometry.coordinates'] = building_df['geometry.coordinates'].apply(lambda x : make_pol(x))
building_df.rename(columns={'geometry.coordinates' : 'geometry'}, inplace=True)
# building_df 데이터프레임을 GeoDataFrame으로 변환
building_df = gpd.GeoDataFrame(building_df, geometry='geometry')
building_df = building_df[[]]

In [12]:
building_df

,type,properties.BUL_MAN_NO,properties.SIG_CD,properties.RN_CD,properties.RDS_MAN_NO,properties.BSI_INT_SN,properties.EQB_MAN_SN,properties.BULD_SE_CD,properties.BULD_MNNM,properties.BULD_SLNO,properties.BULD_NM,properties.BUL_ENG_NM,properties.BULD_NM_DC,properties.BDTYP_CD,properties.BUL_DPN_SE,properties.POS_BUL_NM,properties.EMD_CD,properties.LI_CD,properties.MNTN_YN,properties.LNBR_MNNM,properties.LNBR_SLNO,properties.NTFC_DE,properties.MVM_RES_CD,properties.MVMN_RESN,properties.MVMN_DE,properties.OPERT_DE,properties.BSI_ZON_NO,properties.BD_MGT_SN,properties.GRO_FLO_CO,properties.UND_FLO_CO,properties.RDS_SIG_CD,geometry.type,geometry
0,Feature,35151,11560,3118015,3133,22443,0,0,74,0,None,None,None,03001,M,None,132,00,0,3906,0,20161124,31,None,20161124,20161118132310,7431,1156013200139060000000001,2,0,11560,Polygon,"POLYGON ((126.91438 37.49991, 126.91438 37.499..."
1,Feature,36474,11560,4154180,1275,7723,0,0,21,0,다온채,None,None,01002,M,다온채,133,00,0,778,10,20180412,76,건축물대장건물명변경,20210209,20210209171816,7378,1156013300107780010000878,6,0,11560,Polygon,"POLYGON ((126.89548 37.49940, 126.89554 37.499..."
2,Feature,36475,11560,4154741,1240,7551,0,0,16,0,None,None,None,01002,M,None,132,00,0,97,62,20180412,31,None,20180412,20180412085203,7318,1156013200100970062016633,5,0,11560,Polygon,"POLYGON ((126.92025 37.51485, 126.92028 37.514..."
3,Feature,39673,11560,3118017,1353,8136,0,0,117,4,None,None,None,03001,M,None,128,00,0,101,8,20211007,31,None,20211007,20211008143423,7208,1156012800101010008019687,4,0,11560,Polygon,"POLYGON ((126.89428 37.53735, 126.89425 37.537..."
4,Feature,39671,11560,4154472,932,5781,0,0,10,8,None,None,None,04402,M,None,127,00,0,33,19,20211007,31,None,20211007,20211008143419,7269,1156012700100330019020700,8,1,11560,Polygon,"POLYGON ((126.88563 37.52688, 126.88564 37.526..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25009,Feature,37041,11560,4154669,1776,10948,0,0,24,15,None,None,None,02003,M,None,132,00,0,463,11,20181220,31,None,20181220,20181220084950,7356,1156013200104630011018356,5,0,11560,Polygon,"POLYGON ((126.92127 37.50947, 126.92131 37.509..."
25010,Feature,37042,11560,4154397,2593,16065,0,0,1,0,None,None,None,01002,M,None,133,00,0,894,37,20181220,31,None,20181220,20181220084950,7428,1156013300108940037004980,4,1,11560,Polygon,"POLYGON ((126.90556 37.49481, 126.90553 37.494..."
25011,Feature,35150,11560,3118015,3133,22444,0,0,76,0,신길동3902 상가,None,None,03001,M,신길동3902상가,132,00,0,3902,0,20161124,76,건축물대장건물명변경,20200812,20200812130915,7431,1156013200139020000000001,2,0,11560,Polygon,"POLYGON ((126.91461 37.49990, 126.91460 37.499..."
25012,Feature,35171,11560,4154456,342,2080,0,0,20,6,블루움,None,None,02003,M,블루움,128,00,0,247,1,20161201,76,건축물대장건물명변경,20161201,20170825165650,7206,1156012800102470001019627,6,0,11560,Polygon,"POLYGON ((126.89652 37.53799, 126.89663 37.537..."


In [ ]:
# 맵 객체 생성 및 데이터 로드
ydp_building_map = KeplerGl(height=1000, width=1500)
ydp_building_map.add_data(data=building_df, name="영등포구 건물 데이터")
ydp_building_map.add_data(data=ydp_gdf, name="영등포구 행정동 데이터")
ydp_building_map.add_data(data=roadname_df, name="영등포구 도로망 데이터")

# 맵 출력 및 상세설정
ydp_building_map

In [ ]:
# 맵 저장
ydp_building_map.save_to_html(file_name="visualization/영등포구 건물 및 소방관련/영등포구 소방시설 시각화 map.html")

Map saved to visualization/영등포구 건물 및 소방관련/영등포구 소방시설 시각화 map.html!


In [10]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_001/12.영등포_건물노후도.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
building_oldness_df = pd.json_normalize(geojson_data['features'])
building_oldness_df['geometry.coordinates'] = building_oldness_df['geometry.coordinates'].apply(lambda x : make_pol(x))
building_oldness_df.rename(columns={'geometry.coordinates' : 'geometry'}, inplace=True)
# building_oldness_df 데이터프레임을 GeoDataFrame으로 변환
building_oldness_df = gpd.GeoDataFrame(building_oldness_df, geometry='geometry')

In [11]:
building_oldness_df

,type,properties.pnu,properties.emd_nm,properties.bld_nm,properties.dong_nm,properties.gfa,properties.strct_nm,properties.usage_nm,properties.useapr_day,properties.old_year,geometry.type,geometry
0,Feature,1156011600102370031,서울특별시 영등포구 당산동6가,None,None,182.3,벽돌구조,단독주택,1991-08-09,34,Polygon,"POLYGON ((126.90319 37.53383, 126.90315 37.533..."
1,Feature,1156012900100260003,서울특별시 영등포구 양평동5가,M타워,None,1450.6,철근콘크리트구조,제1종근린생활시설,2022-09-07,3,Polygon,"POLYGON ((126.89281 37.53805, 126.89281 37.538..."
2,Feature,1156012800100640003,서울특별시 영등포구 양평동4가,인투타워,None,1554.54,철근콘크리트구조,제2종근린생활시설,2022-09-13,3,Polygon,"POLYGON ((126.89696 37.53617, 126.89700 37.536..."
3,Feature,1156013300106850014,서울특별시 영등포구 대림동,None,None,190.15,철근콘크리트구조,단독주택,2022-09-22,3,Polygon,"POLYGON ((126.89998 37.49983, 126.89998 37.499..."
4,Feature,1156013200148260000,서울특별시 영등포구 신길동,None,None,762.64,철근콘크리트구조,제2종근린생활시설,2022-09-22,3,Polygon,"POLYGON ((126.90957 37.49332, 126.90957 37.493..."
...,...,...,...,...,...,...,...,...,...,...,...,...
21480,Feature,1156011500100420000,서울특별시 영등포구 당산동5가,당산 삼성 래미안,413동,6858.38,철근콘크리트구조,공동주택,2003-12-30,22,Polygon,"POLYGON ((126.90364 37.53158, 126.90336 37.531..."
21481,Feature,1156011500100420000,서울특별시 영등포구 당산동5가,당산 삼성 래미안,410동,8517.83,철근콘크리트구조,공동주택,2003-12-30,22,Polygon,"POLYGON ((126.90267 37.53205, 126.90233 37.532..."
21482,Feature,1156011500100420000,서울특별시 영등포구 당산동5가,당산 삼성 래미안,408동,12233.15,철근콘크리트구조,공동주택,2003-12-30,22,Polygon,"POLYGON ((126.90204 37.53266, 126.90148 37.532..."
21483,Feature,1156011500100420000,서울특별시 영등포구 당산동5가,당산 삼성 래미안,406동,5407.3,철근콘크리트구조,공동주택,2003-12-30,22,Polygon,"POLYGON ((126.90437 37.53187, 126.90408 37.532..."
